## LLM Client test

Day1 작업 중에 LLM 통합 관련 기능 테스트

In [1]:
import json
import sys
from pathlib import Path

from dotenv import load_dotenv
from app.llm import LLMClient

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Load environment variables
load_dotenv(project_root / ".env")

print("✓ Setup complete")

✓ Setup complete


### 1. Basic Chat Completion

기본적인 채팅 완성 기능 테스트

In [2]:
# OpenAI 클라이언트 생성
openai_client = LLMClient(provider="openai", temperature=0.7)

messages = [
    {"role": "system", "content": "You are a helpful AI research assistant."},
    {"role": "user", "content": "2024년 AI 분야 키 트렌드 5가지를 알려줘."},
]

response = openai_client.chat_completion(messages, max_tokens=300)
print("OpenAI Response:")
print(response)

OpenAI Response:
2024년에는 AI 분야에서 몇 가지 주요 트렌드가 주목받을 것으로 예상됩니다. 다음은 그 중 5가지입니다:

1. **생성 AI의 발전과 응용 확대**: 생성형 AI는 텍스트, 이미지, 음악, 비디오 등 다양한 콘텐츠를 생성하는 데 사용됩니다. 이러한 기술의 정확성과 창의성이 더욱 향상되면서 마케팅, 엔터테인먼트, 교육 등 여러 분야에서 활용이 늘어날 것입니다.

2. **AI의 윤리 및 규제 강화**: AI 기술의 급속한 발전과 함께 개인 정보 보호, 편향성, 투명성 등의 윤리적 문제에 대한 논의가 계속될 것입니다. 이에 따라 정부와 기관들이 AI 기술에 대한 규제를 강화하고, AI 시스템의 신뢰성과 책임성을 높이기 위한 노력이 증가할 것입니다.

3. **엣지 AI와 분산 컴퓨팅**: 엣지 컴퓨팅을 통해 데이터를 중앙 서버가 아닌 데이터 생성 지점에서 처리함으로써 실시간 처리와 데이터 프라이버시가 중요한 분야에서 AI의 채택이 증가할 것입니다. 특히 IoT 기기와의 결합이 주목받을 것입니다.

4. **멀티모달 AI의 발전**: 텍스트, 이미지, 오디오 등 다양한 데이터를 동시에 처리하고 이해하는 멀티모달 AI 시스템이 더욱 발전할 것입니다.


### 2. Claude Chat Completion

Claude API를 사용한 채팅 완성을 테스트.

In [3]:
# Claude 클라이언트 생성 (API 키가 설정된 경우에만)
try:
    claude_client = LLMClient(provider="claude", temperature=0.7)

    messages = [{"role": "user", "content": "2024년 AI 분야 키 트렌드 5가지를 알려줘."}]

    response = claude_client.chat_completion(messages, max_tokens=300)
    print("Claude Response:")
    print(response)
except Exception as e:
    print(f"Claude API not available: {e}")

Claude Response:
# 2024년 AI 분야 주요 트렌드 5가지

## 1. **생성형 AI의 실용화 및 기업 도입 가속화**
- ChatGPT, Claude 등 LLM의 기업 업무 통합
- AI 코파일럿(Copilot) 도구들의 확산
- 맞춤형 GPT 및 엔터프라이즈 솔루션 증가

## 2. **멀티모달 AI의 부상**
- 텍스트, 이미지, 음성, 영상을 통합 처리하는 AI
- GPT-4V, Gemini 등 멀티모달 모델 상용화
- 더 자연스러운 인간-AI 상호작용 구현

## 3. **AI 규제 및 윤리 프레임워크 강화**
- EU AI Act 등 글로벌 AI 규제 본격화
- AI 안전성, 투명성, 책임성에 대한 관심 증가


### 3. JSON Response Format

JSON 형식의 응답을 요청하고 파싱.

In [4]:
client = LLMClient(provider="openai")

messages = [
    {"role": "system", "content": "You are a helpful assistant that responds in JSON format."},
    {
        "role": "user",
        "content": """
        기사 타이틀을 분석하고 다음 기준으로 JSOM 포맷으로 정리해줘.:
        - category: one of (paper, news, report)
        - importance_score: 0.0 to 1.0
        - keywords: list of 3-5 relevant keywords
        - summary: brief one-line summary

        Title: "GPT-5 Achieves Human-Level Performance on Complex Reasoning Tasks"
        """,
    },
]

response = client.chat_completion(messages, response_format="json", max_tokens=300)
print("JSON Response:")
print(response)

# Parse JSON
data = json.loads(response)
print("\nParsed Data:")
print(json.dumps(data, indent=2))

JSON Response:
{
    "category": "news",
    "importance_score": 0.9,
    "keywords": ["GPT-5", "human-level performance", "complex reasoning"],
    "summary": "GPT-5 demonstrates human-level capabilities in complex reasoning tasks."
}

Parsed Data:
{
  "category": "news",
  "importance_score": 0.9,
  "keywords": [
    "GPT-5",
    "human-level performance",
    "complex reasoning"
  ],
  "summary": "GPT-5 demonstrates human-level capabilities in complex reasoning tasks."
}


### 4. Article Summarization (Korean)

논문을 한국어로 요약하는 기능을 테스트

In [5]:
client = LLMClient(provider="openai", temperature=0.5)

article_text = """
Researchers at MIT have developed a new neural network architecture
that combines the benefits of transformers with the efficiency of
convolutional neural networks. The hybrid model, dubbed TransConv,
achieves state-of-the-art results on image classification tasks while
requiring 40% less computational resources than traditional transformer
models. The key innovation lies in the selective attention mechanism
that adaptively chooses between local and global feature processing
based on the input characteristics.
"""

messages = [
    {"role": "system", "content": "You are an AI research expert. Summarize articles in Korean."},
    {
        "role": "user",
        "content": f"다음 논문 내용을 한국어로 3-4문장으로 요약해주세요:\n\n{article_text}",
    },
]

summary = client.chat_completion(messages, max_tokens=500)
print("Korean Summary:")
print(summary)

Korean Summary:
MIT 연구진이 새로운 신경망 구조를 개발했습니다. 이 구조는 트랜스포머의 장점과 합성곱 신경망의 효율성을 결합한 하이브리드 모델로, TransConv라고 불립니다. 이 모델은 이미지 분류 작업에서 최첨단 성능을 발휘하면서도 전통적인 트랜스포머 모델보다 40% 적은 계산 자원을 요구합니다. 핵심 혁신은 입력 특성에 따라 지역 및 전역 특징 처리를 선택적으로 수행하는 주의 메커니즘에 있습니다.


### 5. Embedding Generation

텍스트 임베딩을 생성하고 유사도를 계산

In [6]:
import numpy as np

client = LLMClient(provider="openai")

# 여러 텍스트의 임베딩 생성
texts = [
    "Transformer architecture in deep learning",
    "Attention mechanism for neural networks",
    "Reinforcement learning for robotics",
    "Computer vision using CNNs",
]

embeddings = [client.generate_embedding(text) for text in texts]

print(f"Generated {len(embeddings)} embeddings")
print(f"Embedding dimension: {len(embeddings[0])}")
print(f"First embedding (first 5 values): {embeddings[0][:5]}")

Generated 4 embeddings
Embedding dimension: 1536
First embedding (first 5 values): [-0.030603239312767982, -0.06797607243061066, 0.004770813975483179, -0.015896877273917198, 0.019433407112956047]


In [7]:
# 코사인 유사도 계산
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


print("\nCosine Similarities:")
for i, text_i in enumerate(texts):
    for j, text_j in enumerate(texts):
        if i < j:
            similarity = cosine_similarity(embeddings[i], embeddings[j])
            print(f"'{text_i}' vs '{text_j}': {similarity:.4f}")


Cosine Similarities:
'Transformer architecture in deep learning' vs 'Attention mechanism for neural networks': 0.4853
'Transformer architecture in deep learning' vs 'Reinforcement learning for robotics': 0.3040
'Transformer architecture in deep learning' vs 'Computer vision using CNNs': 0.4224
'Attention mechanism for neural networks' vs 'Reinforcement learning for robotics': 0.3288
'Attention mechanism for neural networks' vs 'Computer vision using CNNs': 0.3612
'Reinforcement learning for robotics' vs 'Computer vision using CNNs': 0.3521


### 6. Async Operations

비동기 작업을 테스트

In [8]:
import asyncio


async def test_async_operations():
    client = LLMClient(provider="openai")

    # Async chat completion
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "머신러닝을 한문장으로 정의한다면?"},
    ]

    response = await client.achat_completion(messages, max_tokens=100)
    print("Async Chat Response:")
    print(response)
    print()

    # Async embedding
    embedding = await client.agenerate_embedding("Deep learning for NLP")
    print(f"Async Embedding dimension: {len(embedding)}")
    print(f"First 5 values: {embedding[:5]}")


# Run async function
await test_async_operations()

Async Chat Response:
머신러닝은 컴퓨터가 명시적인 프로그래밍 없이도 데이터로부터 학습하고 예측하거나 결정을 내릴 수 있도록 하는 인공지능의 한 분야입니다.

Async Embedding dimension: 1536
First 5 values: [-0.011855199001729488, -0.02571532316505909, 0.03698211908340454, -0.026695990934967995, -0.014067153446376324]


### 7. Batch Processing

여러 문서를 동시에 처리

In [9]:
async def process_articles_batch(articles):
    """여러 논문을 동시에 요약"""
    client = LLMClient(provider="openai", temperature=0.5)

    async def summarize_one(article):
        messages = [
            {"role": "system", "content": "Summarize the article in Korean in 2 sentences."},
            {"role": "user", "content": article},
        ]
        return await client.achat_completion(messages, max_tokens=200)

    # 모든 요약을 동시에 실행
    summaries = await asyncio.gather(*[summarize_one(article) for article in articles])
    return summaries


# Test batch processing
articles = [
    "GPT-5 shows remarkable improvements in reasoning capabilities.",
    "New transformer architecture reduces computational costs by 40%.",
    "Reinforcement learning breakthrough enables better robot control.",
]

summaries = await process_articles_batch(articles)

print("Batch Processing Results:")
for i, (article, summary) in enumerate(zip(articles, summaries, strict=False), 1):
    print(f"\n{i}. Original: {article}")
    print(f"   Summary: {summary}")

Batch Processing Results:

1. Original: GPT-5 shows remarkable improvements in reasoning capabilities.
   Summary: GPT-5는 추론 능력에서 눈에 띄는 향상을 보였습니다. 이 모델은 복잡한 문제를 해결하고 논리적인 결론을 도출하는 데 있어 이전 버전보다 더 뛰어난 성능을 발휘합니다.

2. Original: New transformer architecture reduces computational costs by 40%.
   Summary: 새로운 트랜스포머 아키텍처는 계산 비용을 40% 절감한다. 이로 인해 더 효율적인 데이터 처리 및 모델 학습이 가능해졌다.

3. Original: Reinforcement learning breakthrough enables better robot control.
   Summary: 강화 학습의 돌파구가 로봇 제어를 더욱 향상시켰습니다. 이 기술은 로봇이 더 복잡한 작업을 효율적으로 수행할 수 있도록 지원합니다.


### 8. Temperature Comparison

다양한 temperature 값에 따른 응답 차이를 비교

In [10]:
temperatures = [0.0, 0.5, 1.0]
prompt = "Explain neural networks in one sentence."

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

print("Temperature Comparison:")
print(f"Prompt: {prompt}\n")

for temp in temperatures:
    client = LLMClient(provider="openai", temperature=temp)
    response = client.chat_completion(messages, max_tokens=100)
    print(f"Temperature {temp}:")
    print(f"  {response}")
    print()

Temperature Comparison:
Prompt: Explain neural networks in one sentence.

Temperature 0.0:
  Neural networks are computational models inspired by the human brain, consisting of interconnected layers of nodes (neurons) that process and learn from data to perform tasks such as classification, regression, and pattern recognition.

Temperature 0.5:
  Neural networks are computational models inspired by the human brain, consisting of interconnected layers of nodes (or "neurons") that process and learn from data through weighted connections and activation functions.

Temperature 1.0:
  Neural networks are computational models inspired by the human brain, consisting of interconnected layers of nodes (neurons) that process input data to recognize patterns and make predictions or decisions.



### 9. Error Handling

에러 처리를 테스트

In [11]:
# Test invalid provider
try:
    invalid_client = LLMClient(provider="invalid")
except ValueError as e:
    print(f"✓ Caught expected error for invalid provider: {e}")

# Test with very large max_tokens (should be handled by API)
try:
    client = LLMClient(provider="openai")
    messages = [{"role": "user", "content": "Hello"}]
    response = client.chat_completion(messages, max_tokens=100000)
    print("\nResponse with large max_tokens received (API handled it)")
except Exception as e:
    print(f"Error with large max_tokens: {e}")

✓ Caught expected error for invalid provider: Unsupported provider: invalid
Error with large max_tokens: LLM completion failed: litellm.BadRequestError: OpenAIException - max_tokens is too large: 100000. This model supports at most 16384 completion tokens, whereas you provided 100000.


### 10. Use Case: Research Article Analysis

실제 사용 사례: 연구 논문 분석 파이프라인

In [12]:
async def analyze_research_article(title, abstract):
    """연구 논문을 분석하고 메타데이터를 추출"""
    client = LLMClient(provider="openai", temperature=0.3)

    # 1. 카테고리 분류 및 중요도 평가
    analysis_messages = [
        {
            "role": "system",
            "content": "You are an AI research expert. Analyze research articles and respond in JSON.",
        },
        {
            "role": "user",
            "content": f"""
            Analyze this research article and return JSON with:
            - category: one of (paper, news, report)
            - importance_score: 0.0 to 1.0
            - keywords: list of 5 keywords
            - field: research field (e.g., "Computer Vision", "NLP", "Robotics")

            Title: {title}
            Abstract: {abstract}
            """,
        },
    ]

    # 2. 한국어 요약
    summary_content = (
        f"다음 논문을 한국어로 3-4문장으로 요약해주세요:\n\n" f"Title: {title}\n\nAbstract: {abstract}"
    )
    summary_messages = [
        {"role": "system", "content": "You are an AI research expert. Summarize in Korean."},
        {
            "role": "user",
            "content": summary_content,
        },
    ]

    # 3. 임베딩 생성
    embedding_text = f"{title} {abstract}"

    # 모두 비동기로 동시 실행
    analysis_task = client.achat_completion(analysis_messages, response_format="json", max_tokens=300)
    summary_task = client.achat_completion(summary_messages, max_tokens=500)
    embedding_task = client.agenerate_embedding(embedding_text)

    analysis_response, summary, embedding = await asyncio.gather(
        analysis_task, summary_task, embedding_task
    )

    # 결과 파싱
    analysis = json.loads(analysis_response)

    return {
        "title": title,
        "category": analysis.get("category"),
        "importance_score": analysis.get("importance_score"),
        "keywords": analysis.get("keywords", []),
        "field": analysis.get("field"),
        "summary_korean": summary,
        "embedding_dim": len(embedding),
        "embedding_preview": embedding[:5],
    }


# Test the analysis pipeline
test_article = {
    "title": "Attention Is All You Need",
    "abstract": """
    The dominant sequence transduction models are based on complex recurrent or
    convolutional neural networks in an encoder-decoder configuration. The best
    performing models also connect the encoder and decoder through an attention
    mechanism. We propose a new simple network architecture, the Transformer,
    based solely on attention mechanisms, dispensing with recurrence and convolutions
    entirely.
    """,
}

result = await analyze_research_article(test_article["title"], test_article["abstract"])

print("Research Article Analysis Result:")
print(json.dumps(result, indent=2, ensure_ascii=False))

Research Article Analysis Result:
{
  "title": "Attention Is All You Need",
  "category": "paper",
  "importance_score": 0.95,
  "keywords": [
    "Transformer",
    "attention mechanism",
    "sequence transduction",
    "encoder-decoder",
    "neural networks"
  ],
  "field": "NLP",
  "summary_korean": "\"Attention Is All You Need\" 논문은 기존의 복잡한 순환 신경망이나 합성곱 신경망 기반의 인코더-디코더 모델 대신, 주의 메커니즘만을 활용한 새로운 간단한 네트워크 아키텍처인 트랜스포머를 제안합니다. 이 모델은 순환과 합성곱을 완전히 배제하고, 인코더와 디코더를 주의 메커니즘을 통해 연결하여 성능을 향상시킵니다.",
  "embedding_dim": 1536,
  "embedding_preview": [
    0.020583556964993477,
    -0.007367096375674009,
    -0.016429178416728973,
    0.009078700095415115,
    0.02123717963695526
  ]
}


### 11. Streaming Response

스트리밍 방식으로 실시간 응답을 받는 기능을 테스트

In [13]:
# Test streaming response
client = LLMClient(provider="openai", temperature=0.7)

messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "AI 기술의 미래에 대해 3문장으로 설명해줘."},
]

print("Streaming Response:")
print("-" * 50)

# stream=True로 호출하면 LiteLLM generator가 반환됨
response_stream = client.chat_completion(messages, max_tokens=200, stream=True)

# 사용자가 직접 stream을 처리
full_response = ""
for chunk in response_stream:
    # LiteLLM의 streaming chunk 구조에서 content 추출
    if chunk.choices[0].delta.content:
        text = chunk.choices[0].delta.content
        print(text, end="", flush=True)
        full_response += text

print("\n" + "-" * 50)
print(f"\nFull response length: {len(full_response)} characters")

Streaming Response:
--------------------------------------------------
AI 기술의 미래는 더욱 정교하고 인간과 비슷한 수준의 이해 및 의사소통 능력을 갖춘 시스템의 개발로 이어질 것입니다. 이러한 기술은 의료, 교육, 교통 등 다양한 분야에서 혁신을 가져와 생산성을 높이고 삶의 질을 향상시킬 것입니다. 그러나 AI의 발전은 윤리적 문제와 개인정보 보호 등의 새로운 도전을 동반할 것이며, 이에 대한 사회적 합의와 법적 규제가 필요할 것입니다.
--------------------------------------------------

Full response length: 209 characters


In [14]:
# Test async streaming response
async def test_async_streaming():
    client = LLMClient(provider="openai", temperature=0.7)

    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": "머신러닝과 딥러닝의 차이를 간단히 설명해줘."},
    ]

    print("\nAsync Streaming Response:")
    print("-" * 50)

    # stream=True로 호출하면 LiteLLM async generator가 반환됨
    response_stream = await client.achat_completion(messages, max_tokens=200, stream=True)

    # 사용자가 직접 async stream을 처리
    full_response = ""
    async for chunk in response_stream:
        # LiteLLM의 streaming chunk 구조에서 content 추출
        if chunk.choices[0].delta.content:
            text = chunk.choices[0].delta.content
            print(text, end="", flush=True)
            full_response += text

    print("\n" + "-" * 50)
    print(f"\nFull response length: {len(full_response)} characters")


await test_async_streaming()


Async Streaming Response:
--------------------------------------------------
머신러닝과 딥러닝은 둘 다 인공지능의 하위 분야로, 데이터로부터 학습하여 작업을 수행하는 기술입니다. 하지만 둘 사이에는 몇 가지 중요한 차이점이 있습니다.

1. **구조**:
   - **머신러닝**: 주로 선형 회귀, 로지스틱 회귀, 결정 트리, 서포트 벡터 머신(SVM), K-평균 군집화 등 다양한 알고리즘을 포함합니다. 데이터의 특성을 수동으로 추출하고 선택하는 과정을 많이 요구합니다.
   - **딥러닝**: 인공신경망을 기반으로 한 머신러닝의 하위 분야입니다. 특히 여러 층을 가진 심층 신경망(Deep Neural Networks)을 사용하여 데이터로부터 자동으로 특징을 학습합니다.

2. **데이터 요구량**:
   - **머신러닝**: 상대
--------------------------------------------------

Full response length: 371 characters


## Summary

이 노트북에서 다룬 내용:

1. ✓ Basic chat completion (OpenAI & Claude)
2. ✓ JSON response format
3. ✓ Korean summarization
4. ✓ Embedding generation and similarity
5. ✓ Async operations
6. ✓ Batch processing
7. ✓ Temperature comparison
8. ✓ Error handling
9. ✓ Complete research article analysis pipeline
10. ✓ Research article analysis pipeline
11. ✓ Streaming response (sync & async)

LLM 클라이언트가 정상적으로 작동하며, 다양한 사용 사례에 활용할 수 있습니다.